In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.insert(0,"/Users/erichsato/Documents/WFP/01_programming/callclarity")
sys.path.insert(0,"/Users/erichsato/Documents/WFP/01_programming/nlp_cfm")
os.environ["PROJECT_ABSPATH"] = "/Users/erichsato/Documents/WFP/01_programming/nlp_cfm"

import numpy as np
import pandas as pd

from callclarity.error_report.detectors.exact_incoherence import ExactIncoherenceDetector
from callclarity.error_report.detectors.approx_incoherence import ApproximateIncoherenceDetector
from callclarity.error_report.detectors.model_incoherence import ModelIncoherenceDetector
from callclarity.error_report.embeddings.embedder import TextEmbedder
from callclarity.error_report.sorters.semantic_sorter import SemanticSorter
from callclarity.error_report.inspector import IncoherenceInspector
from callclarity.error_report.reporter import IncoherenceReporter
from ftm.io import dio

dio.set_run_id("data",run_id = "2023-11-10_corrected_ukr")
dio.set_run_id("model",run_id = "2023-11-10_corrected_ukr")

In [8]:
# df = dio.read_data("domain","crm",path_args = {"country":"ukr"})

df = (
    dio.read_data("raw","crm_ukr_07")
    .merge(
        dio.read_data("raw","crm_ukr_07_translations")[["Case ID (Number)","description_translated"]],
        on = ["Case ID (Number)"],
        how = "left"
    )
    
)
df["id"] = df["Case ID (Number)"]
df["raw_descrip""tion"] = df["description_translated"]
df["raw_mcat_cat_category"] = df["Macro Category"] + " - " + df["Category"]
df = df[df["raw_mcat_cat_category"].notnull()]

2024-01-02 10:46:57.224 | INFO     | ftm.io:_read_excel:412 - Reading excel from '/Users/erichsato/Documents/WFP/01_programming/nlp_cfm/data/raw/crm/ukr/NLP Request_September 2023.xlsx'
2024-01-02 10:47:02.193 | INFO     | ftm.io:_read_excel:412 - Reading excel from '/Users/erichsato/Documents/WFP/01_programming/nlp_cfm/data/raw/crm/ukr/NLP Request_September 2023_translations.xlsx'


# Exact Incoherence

In [3]:
eid = ExactIncoherenceDetector(df["raw_description"],df["raw_mcat_cat_category"])

In [4]:
error_flags, error_recs, error_metadata = eid.get_errors()

2023-12-26 10:41:18.615 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:30 - Calculating 'exact_incoherence' conflicts
2023-12-26 10:41:18.631 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:42 -  └ Filtering conflicted cases
2023-12-26 10:41:18.632 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:46 -  └ Getting cluster indices
2023-12-26 10:41:18.632 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:50 -  └ Getting error recommendations
2023-12-26 10:41:18.634 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:57 -  └ Getting error flags


In [5]:
np.sum(error_flags)

7390

In [6]:
df.shape[0] - np.sum(error_flags)

2950

# Approx Incoherence 

In [7]:
from sklearn.neighbors import radius_neighbors_graph
from scipy.sparse.csgraph import connected_components
from scipy.sparse import csr_matrix

In [8]:
embedder = TextEmbedder(df["raw_description"])
embeddings = embedder.embed()

2023-12-26 10:41:18.876 | INFO     | callclarity.utils.device:get_torch_device:26 - Using Apple Silicon as the torch device


Batches:   0%|          | 0/81 [00:00<?, ?it/s]

In [10]:
len(embeddings)

10340

In [11]:
len(df["raw_mcat_cat_category"])

10340

In [13]:
aid = ApproximateIncoherenceDetector(embeddings = embeddings.tolist(), labels = df["raw_mcat_cat_category"])

In [14]:
error_flags, error_recs, error_metadata = aid.get_errors()

2023-12-26 10:43:28.930 | INFO     | callclarity.error_report.detectors.approx_incoherence:get_errors:57 -  └ Calculating clusters
2023-12-26 10:43:32.417 | INFO     | callclarity.error_report.detectors.approx_incoherence:get_errors:61 -    └ 3,905 clusters were found.
2023-12-26 10:43:32.417 | INFO     | callclarity.error_report.detectors.approx_incoherence:get_errors:64 - Calculating 'approx_incoherence' conflicts
2023-12-26 10:43:32.927 | INFO     | callclarity.error_report.detectors.approx_incoherence:get_errors:76 -  └ Filtering conflicted cases
2023-12-26 10:43:32.928 | INFO     | callclarity.error_report.detectors.approx_incoherence:get_errors:80 -  └ Getting cluster indices
2023-12-26 10:43:32.928 | INFO     | callclarity.error_report.detectors.approx_incoherence:get_errors:84 -  └ Getting error recommendations
2023-12-26 10:43:32.929 | INFO     | callclarity.error_report.detectors.approx_incoherence:get_errors:91 -  └ Getting error flags


In [ ]:
for nrows in [100,500,1000,5000,10000,50000,100000]:
    
    print(f"{nrows} rows")
    %time nn_graph = radius_neighbors_graph(embeddings[:nrows],radius = 0.1, mode = "distance", metric = "cosine", n_jobs = -1)
    print()

# Model Incoherence 

In [16]:
mid = ModelIncoherenceDetector(embeddings = embeddings.tolist(), labels = df["raw_mcat_cat_category"])

In [17]:
error_flags, error_recs, error_metadata = mid.get_errors()

2023-12-26 10:43:57.531 | INFO     | callclarity.error_report.detectors.model_incoherence:get_errors:58 -  └ Calculating clusters
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/count.py:1022: UserWarning: Duplicated some data across multiple folds to ensure training does not fail because these classes do not have enough data for proper cross-validation: {34}.
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/count.py:1022: UserWarning: Duplicated some data across multiple folds to ensure training does not fail because these classes do not have enough data for proper cross-validation: {37}.
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/count.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/filter.py:907: UserWarning: May not flag all label issues in class: 10, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/filter.py:907: UserWarning: May not flag all label issues in class: 12, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/filter.py:907: UserWarning: May not flag all label issues in class: 0, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/filter.py:907: UserWarning: May not flag all label issues in class: 1, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/clea

In [137]:
len(error_flags)

10340

In [138]:
len(error_recs)

10340

In [321]:
np.sum(error_flags)

1461

# Semantic sorter 

In [632]:
semantic_sorter = SemanticSorter(embeddings, embedding_distance_step=0.01)

2023-12-20 18:47:59.698 | INFO     | callclarity.error_report.sorters.semantic_sorter:_init_nn_graph:24 - Instantiating radius neighbors graph


In [631]:
sorted_indices = semantic_sorter.get_sort_indices()

100%|███████████████████████████████████████████| 80/80 [00:02<00:00, 27.59it/s]


In [629]:
n = 6000
df.iloc[sorted_indices].iloc[n:n+10]

,country,id,date_created,raw_description,raw_macro_category,raw_category,raw_subcategory,raw_mcat_cat_category,macro_category,category,mcat_cat_category,grouped_categories_list,subcategory
9310,ukr,612303,2023-09-11 06:43:00,products,Request for assistance,Food,<NA>,Request for assistance - Food,Grouped others,Grouped others,Grouped others - Grouped others,"[Other - Other (please explain), Allegations of misconduct - Fraud, Request for assistance - Cash, Allegations of misconduct - Disrespectful treatment, Request for assistance - Food, Allegations of misconduct - Diversion of assistance, Allegations of misconduct - Discrimination, Request for assistance - Other (please explain), Allegations of misconduct - Corruption, Request for specialised services (non-WFP) - Other (please explain), Allegations of misconduct - Other (please explain), Request for specialised services (non-WFP) - Medical, Request for specialised services (non-WFP) - Legal]",<NA>
9633,ukr,613281,2023-09-11 14:44:00,products,Request for assistance,Food,<NA>,Request for assistance - Food,Grouped others,Grouped others,Grouped others - Grouped others,"[Other - Other (please explain), Allegations of misconduct - Fraud, Request for assistance - Cash, Allegations of misconduct - Disrespectful treatment, Request for assistance - Food, Allegations of misconduct - Diversion of assistance, Allegations of misconduct - Discrimination, Request for assistance - Other (please explain), Allegations of misconduct - Corruption, Request for specialised services (non-WFP) - Other (please explain), Allegations of misconduct - Other (please explain), Request for specialised services (non-WFP) - Medical, Request for specialised services (non-WFP) - Legal]",<NA>
8157,ukr,596131,2023-08-25 06:12:00,Outgoing call\ndoes not meet the,Request for information,Targeting criteria,<NA>,Request for information - Targeting criteria,Request for information,Targeting criteria,Request for information - Targeting criteria,[],<NA>
8165,ukr,596191,2023-08-25 06:46:00,Outgoing call\ndoes not meet the,Request for information,Targeting criteria,<NA>,Request for information - Targeting criteria,Request for information,Targeting criteria,Request for information - Targeting criteria,[],<NA>
8178,ukr,596480,2023-08-25 07:37:00,Outgoing call\ndoes not meet the,Request for information,Targeting criteria,<NA>,Request for information - Targeting criteria,Request for information,Targeting criteria,Request for information - Targeting criteria,[],<NA>
8223,ukr,598297,2023-08-25 10:18:00,Outgoing call\ndoes not meet the,Request for information,Targeting criteria,<NA>,Request for information - Targeting criteria,Request for information,Targeting criteria,Request for information - Targeting criteria,[],<NA>
8232,ukr,598350,2023-08-25 10:35:00,Outgoing call\ndoes not meet the,Request for information,Targeting criteria,<NA>,Request for information - Targeting criteria,Request for information,Targeting criteria,Request for information - Targeting criteria,[],<NA>
8235,ukr,598386,2023-08-25 10:45:00,Outgoing call\nDoes not meet the,Request for information,Targeting criteria,<NA>,Request for information - Targeting criteria,Request for information,Targeting criteria,Request for information - Targeting criteria,[],<NA>
8241,ukr,598463,2023-08-25 11:03:00,Outgoing call\ndoes not meet the,Request for information,Targeting criteria,<NA>,Request for information - Targeting criteria,Request for information,Targeting criteria,Request for information - Targeting criteria,[],<NA>
8245,ukr,598497,2023-08-25 11:15:00,Outgoing call\ndoes not meet the,Request for information,Targeting criteria,<NA>,Request for information - Targeting criteria,Request for information,Targeting criteria,Request for information - Targeting criteria,[],<NA>


# Inspector 

In [3]:
from callclarity.error_report.inspector import IncoherenceInspector

In [106]:
embedder = TextEmbedder(df["raw_description"])
embeddings = embedder.embed().tolist()

2023-12-27 15:50:33.000 | INFO     | callclarity.utils.device:get_torch_device:26 - Using Apple Silicon as the torch device


Batches:   0%|          | 0/81 [00:00<?, ?it/s]

In [127]:
inspector = IncoherenceInspector(
    ids = df["id"],
    texts = df["raw_description"],
    labels = df["raw_mcat_cat_category"],
    embeddings = embeddings,
)
df_inspected = inspector.inspect()

2023-12-27 16:41:50.834 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:58 - Calculating 'exact_incoherence' conflicts
2023-12-27 16:41:51.422 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:70 -  └ Filtering conflicted cases
2023-12-27 16:41:51.423 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:74 -  └ Getting cluster indices
2023-12-27 16:41:51.424 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:78 -  └ Getting error recommendations
2023-12-27 16:41:51.425 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:82 -  └ Getting incoherence top frequency
2023-12-27 16:41:51.426 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:92 -  └ Getting error flags
2023-12-27 16:41:51.504 | INFO     | callclarity.error_report.detectors.approx_incoherence:get_errors:90 -  └ Calculating clusters
2023-12-27 16:41:54.430 | INFO     | call

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/filter.py:907: UserWarning: May not flag all label issues in class: 10, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/filter.py:907: UserWarning: May not flag all label issues in class: 12, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/filter.py:907: UserWarning: May not flag all label issues in class: 0, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/filter.py:907: UserWarning: May not flag all label issues in class: 1, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/clea

In [11]:
inspector = IncoherenceInspector(
    ids = df["id"],
    texts = df["raw_description"],
    labels = df["raw_mcat_cat_category"],
)
df_inspected = inspector.inspect()

2024-01-02 10:48:48.260 | INFO     | callclarity.error_report.inspector:_init_embeddings:46 - Embedding texts
2024-01-02 10:48:48.261 | INFO     | callclarity.utils.device:get_torch_device:26 - Using Apple Silicon as the torch device


Batches:   0%|          | 0/728 [00:00<?, ?it/s]

2024-01-02 10:49:27.543 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:58 - Calculating 'exact_incoherence' conflicts
2024-01-02 10:49:32.225 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:70 -  └ Filtering conflicted cases
2024-01-02 10:49:32.227 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:74 -  └ Getting cluster indices
2024-01-02 10:49:32.228 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:78 -  └ Getting error recommendations
2024-01-02 10:49:32.229 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:82 -  └ Getting incoherence top frequency
2024-01-02 10:49:32.230 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:92 -  └ Getting error flags
2024-01-02 10:49:32.860 | INFO     | callclarity.error_report.detectors.approx_incoherence:get_errors:90 -  └ Calculating clusters
2024-01-02 10:51:35.957 | INFO     | call

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/filter.py:907: UserWarning: May not flag all label issues in class: 2, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/filter.py:907: UserWarning: May not flag all label issues in class: 3, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/filter.py:907: UserWarning: May not flag all label issues in class: 4, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/filter.py:907: UserWarning: May not flag all label issues in class: 5, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanl

In [13]:
df_inspected.shape

(93082, 28)

In [18]:
df_inspected["priority"].fillna(-1).value_counts(normalize = True).sort_index()

-1.0    0.561913
 0.0    0.208676
 1.0    0.155540
 2.0    0.023409
 3.0    0.050461
Name: priority, dtype: float64

# Generate the report 

In [128]:
reporter = IncoherenceReporter(
    df = df_inspected,
    path = "/Users/erichsato/Documents/WFP/01_programming/nlp_cfm/data/temp/report_tests/report_v4.xlsx",
    text_col="text",
    label_col="label",
)
reporter.report()

In [116]:
df_inspected df_inspected["exact_incoherence_metadata"]

SyntaxError: invalid syntax (3621103058.py, line 1)

In [22]:
df_inspected[(df_inspected["priority"].notnull() | df_inspected["cluster_priority"].notnull())]

,id,priority,cluster_priority,text,label,label_recommended,exact_incoherence_recommendation,approx_incoherence_recommendation,model_incoherence_recommendation,is_incoherent,is_exact_incoherence,is_approx_incoherence,is_model_incoherence,exact_incoherence_metadata,approx_incoherence_metadata,model_incoherence_metadata,cluster_incoherence_count,cluster_total_count
4567,559637,0.0,0.0,The MTCN code did not arrive. More than 50 day...,Request for information - Assistance timing or...,Request for information - Targeting criteria,Request for information - Targeting criteria,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,1,1,1,1,"{'exact_incoherence_cluster_index': 165.0, 'ex...","{'approx_incoherence_cluster_index': 0.0, 'app...",{'model_incoherence_severity': 0.6872487681548...,576.0,1393.0
6736,585890,0.0,0.0,The MTCN code did not arrive. Received an SMS ...,Request for information - Assistance duration,Request for information - Targeting criteria,Request for information - Targeting criteria,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,1,1,1,1,"{'exact_incoherence_cluster_index': 68.0, 'exa...","{'approx_incoherence_cluster_index': 0.0, 'app...",{'model_incoherence_severity': 0.9989767111168...,576.0,1393.0
8342,600348,0.0,0.0,The MTCN code did not arrive. Received an SMS ...,Request for information - Other (please explain),Request for information - Targeting criteria,Request for information - Targeting criteria,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,1,1,1,1,"{'exact_incoherence_cluster_index': 68.0, 'exa...","{'approx_incoherence_cluster_index': 0.0, 'app...",{'model_incoherence_severity': 0.9954472010089...,576.0,1393.0
4952,563819,1.0,0.0,The MTCN code did not arrive. More than 50 day...,Request for information - Other (please explain),Complaint - Assistance / Delivery Mechanisms /...,None,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,1,0,1,1,"{'exact_incoherence_cluster_index': nan, 'exac...","{'approx_incoherence_cluster_index': 0.0, 'app...",{'model_incoherence_severity': 0.9883657994091...,576.0,1393.0
6935,540414,1.0,0.0,The MTCN code did not arrive. More than 50 day...,Complaint - Incorrect records,Complaint - Assistance / Delivery Mechanisms /...,None,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,1,0,1,1,"{'exact_incoherence_cluster_index': nan, 'exac...","{'approx_incoherence_cluster_index': 0.0, 'app...",{'model_incoherence_severity': 0.9675125838860...,576.0,1393.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10222,496454,3.0,NaN,did not receive 3 help\nthe burned person's pa...,Complaint - Incorrect records,Complaint - Assistance / Delivery Mechanisms /...,None,None,Complaint - Assistance / Delivery Mechanisms /...,1,0,0,1,"{'exact_incoherence_cluster_index': nan, 'exac...","{'approx_incoherence_cluster_index': nan, 'app...",{'model_incoherence_severity': 0.8422582085989...,NaN,NaN
10245,530047,3.0,NaN,did not receive any payment due to the fact th...,Technical requests - Withdrawal of personally ...,Complaint - Assistance / Delivery Mechanisms /...,None,None,Complaint - Assistance / Delivery Mechanisms /...,1,0,0,1,"{'exact_incoherence_cluster_index': nan, 'exac...","{'approx_incoherence_cluster_index': nan, 'app...",{'model_incoherence_severity': 0.8987482142804...,NaN,NaN
10267,527001,3.0,NaN,"On the website, the status ""Your application h...",Technical requests - Withdrawal of personally ...,Complaint - Assistance / Delivery Mechanisms /...,None,None,Complaint - Assistance / Delivery Mechanisms /...,1,0,0,1,"{'exact_incoherence_cluster_index': nan, 'exac...","{'approx_incoherence_cluster_index': nan, 'app...",{'model_incoherence_severity': 0.7726238277221...,NaN,NaN
10287,535840,3.0,NaN,According to the instru

In [10]:
df_inspected

,id,priority,cluster_priority,text,label,label_recommended,exact_incoherence_recommendation,approx_incoherence_recommendation,model_incoherence_recommendation,is_incoherent,is_exact_incoherence,is_approx_incoherence,is_model_incoherence,exact_incoherence_metadata,approx_incoherence_metadata,model_incoherence_metadata,cluster_incoherence_count,cluster_total_count
4567,559637,0.0,0.0,The MTCN code did not arrive. More than 50 day...,Request for information - Assistance timing or...,Request for information - Targeting criteria,Request for information - Targeting criteria,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,1,1,1,1,"{'exact_incoherence_cluster_index': 165.0, 'ex...","{'approx_incoherence_cluster_index': 0.0, 'app...",{'model_incoherence_severity': 0.7069928319065...,576.0,1393.0
6736,585890,0.0,0.0,The MTCN code did not arrive. Received an SMS ...,Request for information - Assistance duration,Request for information - Targeting criteria,Request for information - Targeting criteria,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,1,1,1,1,"{'exact_incoherence_cluster_index': 68.0, 'exa...","{'approx_incoherence_cluster_index': 0.0, 'app...",{'model_incoherence_severity': 0.9987864743619...,576.0,1393.0
8342,600348,0.0,0.0,The MTCN code did not arrive. Received an SMS ...,Request for information - Other (please explain),Request for information - Targeting criteria,Request for information - Targeting criteria,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,1,1,1,1,"{'exact_incoherence_cluster_index': 68.0, 'exa...","{'approx_incoherence_cluster_index': 0.0, 'app...",{'model_incoherence_severity': 0.9947342570185...,576.0,1393.0
4952,563819,1.0,0.0,The MTCN code did not arrive. More than 50 day...,Request for information - Other (please explain),Complaint - Assistance / Delivery Mechanisms /...,None,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,1,0,1,1,"{'exact_incoherence_cluster_index': nan, 'exac...","{'approx_incoherence_cluster_index': 0.0, 'app...",{'model_incoherence_severity': 0.9890000441534...,576.0,1393.0
6935,540414,1.0,0.0,The MTCN code did not arrive. More than 50 day...,Complaint - Incorrect records,Complaint - Assistance / Delivery Mechanisms /...,None,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,1,0,1,1,"{'exact_incoherence_cluster_index': nan, 'exac...","{'approx_incoherence_cluster_index': 0.0, 'app...",{'model_incoherence_severity': 0.9702969878089...,576.0,1393.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10325,561546,NaN,NaN,"Under number 380962139303, there is no informa...",Complaint - Incorrect records,None,None,None,None,0,0,0,0,"{'exact_incoherence_cluster_index': nan, 'exac...","{'approx_incoherence_cluster_index': nan, 'app...",{'model_incoherence_severity': nan},NaN,NaN
10326,538519,NaN,NaN,According to case 179681\n\nThe person has pro...,Complaint - Physical access challenges,None,None,None,None,0,0,0,0,"{'exact_incoherence_cluster_index': nan, 'exac...","{'approx_incoherence_cluster_index': nan, 'app...",{'model_incoherence_severity': nan},NaN,NaN
10327,552764,NaN,NaN,A person with reduced mobility,Complaint - Physical access challenges,None,None,None,None,0,0,0,0,"{'exact_incoherence_cluster_index': nan, 'exac...","{'approx_incoherence_cluster_index': nan, 'app...",{'model_incoherence_severity': nan},NaN,NaN
10328,537334,NaN,NaN,a person with a disability cannot receive assi...,Complaint - Physical access challenges,None,None,None,None,0,0,0,0,"{'exact_incoherence_cluster_index': nan, 'exac...","{'approx_incoherence_cluster_index': nan, 'app...",{'model_incoherence_severity': nan},NaN,NaN


In [56]:
df_inspected

,id,text,label,embedding,is_exact_incoherence,exact_incoherence_recommendation,exact_incoherence_metadata,is_approx_incoherence,approx_incoherence_recommendation,approx_incoherence_metadata,is_model_incoherence,model_incoherence_recommendation,model_incoherence_metadata,priority,cluster_priority,is_incoherent,cluster_incoherence_count
4567,559637,The MTCN code did not arrive. More than 50 day...,Request for information - Assistance timing or...,"[-0.014842264354228973, -0.03156145662069321, ...",1,Request for information - Targeting criteria,"{'exact_incoherence_cluster_index': 165.0, 'ex...",1,Complaint - Assistance / Delivery Mechanisms /...,"{'approx_incoherence_cluster_index': 0.0, 'app...",0,None,{'model_incoherence_severity': nan},0.0,0.0,1,576.0
6736,585890,The MTCN code did not arrive. Received an SMS ...,Request for information - Assistance duration,"[-0.010520915500819683, -0.023126713931560516,...",1,Request for information - Targeting criteria,"{'exact_incoherence_cluster_index': 68.0, 'exa...",1,Complaint - Assistance / Delivery Mechanisms /...,"{'approx_incoherence_cluster_index': 0.0, 'app...",1,Complaint - Assistance / Delivery Mechanisms /...,{'model_incoherence_severity': 0.9985255345577...,0.0,0.0,1,576.0
8342,600348,The MTCN code did not arrive. Received an SMS ...,Request for information - Other (please explain),"[-0.010520915500819683, -0.023126713931560516,...",1,Request for information - Targeting criteria,"{'exact_incoherence_cluster_index': 68.0, 'exa...",1,Complaint - Assistance / Delivery Mechanisms /...,"{'approx_incoherence_cluster_index': 0.0, 'app...",1,Complaint - Assistance / Delivery Mechanisms /...,{'model_incoherence_severity': 0.9967618459160...,0.0,0.0,1,576.0
4952,563819,The MTCN code did not arrive. More than 50 day...,Request for information - Other (please explain),"[-0.012392878532409668, -0.01640527881681919, ...",0,None,"{'exact_incoherence_cluster_index': nan, 'exac...",1,Complaint - Assistance / Delivery Mechanisms /...,"{'approx_incoherence_cluster_index': 0.0, 'app...",1,Complaint - Assistance / Delivery Mechanisms /...,{'model_incoherence_severity': 0.99046117826451},1.0,0.0,1,576.0
6935,540414,The MTCN code did not arrive. More than 50 day...,Complaint - Incorrect records,"[-0.021083777770400047, -0.03303220495581627, ...",0,None,"{'exact_incoherence_cluster_index': nan, 'exac...",1,Complaint - Assistance / Delivery Mechanisms /...,"{'approx_incoherence_cluster_index': 0.0, 'app...",1,Complaint - Assistance / Delivery Mechanisms /...,{'model_incoherence_severity': 0.9688556062587...,1.0,0.0,1,576.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10325,561546,"Under number 380962139303, there is no informa...",Complaint - Incorrect records,"[0.002346019260585308, -0.049770016223192215, ...",0,None,"{'exact_incoherence_cluster_index': nan, 'exac...",0,None,"{'approx_incoherence_cluster_index': nan, 'app...",0,None,{'model_incoherence_severity': nan},NaN,NaN,0,NaN
10326,538519,According to case 179681\n\nThe person has pro...,Complaint - Physical access challenges,"[0.021950630471110344, 0.006401213817298412, -...",0,None,"{'exact_incoherence_cluster_index': nan, 'exac...",0,None,"{'approx_incoherence_cluster_index': nan, 'app...",0,None,{'model_incoherence_severity': nan},NaN,NaN,0,NaN
10327,552764,A person with reduced mobility,Complaint - Physical access challenges,"[-0.028081171214580536, -0.008712985552847385,...",0,None,"{'exact_incoherence_cluster_index': nan, 'exac...",0,None,"{'approx_incoherence_cluster_index': nan, 'app...",0,None,{'model_incoherence_severity': nan},NaN,NaN,0,NaN
10328,537334,a person with a disability cannot receive assi...,Complaint - Physical access challenges,"[-0.0012608394026756287, 0.008399887941777706,...",0,None,"{'exact_incoherence_cluster_index': nan, 'exac...",0,None,"{'approx_incoherence_cluster_index': nan, 'app...",0,None,{'model_incoherence_severity': nan},NaN,NaN,0,NaN


In [39]:
df_inspected[(df_inspected["priority"] == 3) & (df_inspected["cluster_priority"].isnull())]

,id,text,label,embedding,is_exact_incoherence,exact_incoherence_recommendation,exact_incoherence_metadata,is_approx_incoherence,approx_incoherence_recommendation,approx_incoherence_metadata,is_model_incoherence,model_incoherence_recommendation,model_incoherence_metadata,priority,cluster_priority,is_incoherent,cluster_incoherence_count
7509,589484,How to get food assistance,Request for assistance - Food,"[0.010919742286205292, -0.056632790714502335, ...",0,None,"{'exact_incoherence_cluster_index': nan, 'exac...",0,None,"{'approx_incoherence_cluster_index': nan, 'app...",1,Request for information - Assistance type and ...,{'model_incoherence_severity': 0.8470128352107...,3.0,NaN,1,NaN
8910,606559,How to get food assistance,Request for assistance - Food,"[0.010919742286205292, -0.056632790714502335, ...",0,None,"{'exact_incoherence_cluster_index': nan, 'exac...",0,None,"{'approx_incoherence_cluster_index': nan, 'app...",1,Request for information - Assistance type and ...,{'model_incoherence_severity': 0.8470128352107...,3.0,NaN,1,NaN
8216,598233,Outgoing call Not answered\n,Request for information - Other (please explain),"[0.016490323469042778, 0.012342012487351894, 0...",0,None,"{'exact_incoherence_cluster_index': nan, 'exac...",0,None,"{'approx_incoherence_cluster_index': nan, 'app...",1,Request for information - Targeting criteria,{'model_incoherence_severity': 0.7159209437969...,3.0,NaN,1,NaN
8237,598397,Outgoing call Not answered\n,Request for information - Other (please explain),"[0.016490265727043152, 0.012341977097094059, 0...",0,None,"{'exact_incoherence_cluster_index': nan, 'exac...",0,None,"{'approx_incoherence_cluster_index': nan, 'app...",1,Request for information - Targeting criteria,{'model_incoherence_severity': 0.751842926790409},3.0,NaN,1,NaN
8239,598437,Outgoing call Not answered\n,Request for information - Other (please explain),"[0.016490265727043152, 0.012341977097094059, 0...",0,None,"{'exact_incoherence_cluster_index': nan, 'exac...",0,None,"{'approx_incoherence_cluster_index': nan, 'app...",1,Request for information - Targeting criteria,{'model_incoherence_severity': 0.7159210687108...,3.0,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10245,530047,did not receive any payment due to the fact th...,Technical requests - Withdrawal of personally ...,"[0.011644075624644756, 0.011705193668603897, -...",0,None,"{'exact_incoherence_cluster_index': nan, 'exac...",0,None,"{'approx_incoherence_cluster_index': nan, 'app...",1,Complaint - Assistance / Delivery Mechanisms /...,{'model_incoherence_severity': 0.8836796410364...,3.0,NaN,1,NaN
10267,527001,"On the website, the status ""Your application h...",Technical requests - Withdrawal of personally ...,"[0.0037415430415421724, 0.0019116394687443972,...",0,None,"{'exact_incoherence_cluster_index': nan, 'exac...",0,None,"{'approx_incoherence_cluster_index': nan, 'app...",1,Complaint - Assistance / Delivery Mechanisms /...,{'model_incoherence_severity': 0.8068970599945...,3.0,NaN,1,NaN
10282,535674,265661 repeatedly\n\nA person with reduced mob...,Complaint - Physical access challenges,"[-0.03614500164985657, -0.00664135068655014, -...",0,None,"{'exact_incoherence_cluster_index': nan, 'exac...",0,None,"{'approx_incoherence_cluster_index': nan, 'app...",1,Complaint - Assistance / Delivery Mechanisms /...,{'model_incoherence_severity': 0.8528693147466...,3.0,NaN,1,NaN
10287,535840,According to the instructions of ticket 217777...,Complaint - Physical access challenges,"[0.035441432148218155, -0.027964206412434578, ...",0,None,"{'exact_incoherence_cluster_index': nan, 'exac...",0,None,"{'approx_incoherence_cluster_index': nan, 'app...",1,Complaint - Assistance / Delivery Mechanisms /...,{'model_incoherence_severity': 0.8931862779376...,3.0,NaN,1,NaN


In [62]:
", ".join([f'"{c}"' for c in df_inspected.columns.tolist()])

'"id", "text", "label", "embedding", "is_exact_incoherence", "exact_incoherence_recommendation", "exact_incoherence_metadata", "is_approx_incoherence", "approx_incoherence_recommendation", "approx_incoherence_metadata", "is_model_incoherence", "model_incoherence_recommendation", "model_incoherence_metadata", "priority", "cluster_priority", "is_incoherent", "cluster_incoherence_count"'

In [67]:
df_inspected[
    [
        "id", 
        "priority", "cluster_priority",
        "text", "label",
        "exact_incoherence_recommendation", 
        "approx_incoherence_recommendation", 
        "model_incoherence_recommendation", 
        "cluster_incoherence_count",
        "is_incoherent", 
        "is_exact_incoherence","is_approx_incoherence", "is_model_incoherence", 
        "exact_incoherence_metadata", "approx_incoherence_metadata","model_incoherence_metadata", ]
].T

,4567,6736,8342,4952,6935,9478,2765,2829,7582,10187,...,10317,10318,10320,10322,10323,10325,10326,10327,10328,10338
id,559637,585890,600348,563819,540414,514132,534995,522778,503722,554460,...,538556,537795,567952,567885,537750,561546,538519,552764,537334,545421
priority,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cluster_priority,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
text,The MTCN code did not arrive. More than 50 day...,The MTCN code did not arrive. Received an SMS ...,The MTCN code did not arrive. Received an SMS ...,The MTCN code did not arrive. More than 50 day...,The MTCN code did not arrive. More than 50 day...,The MTCN code did not arrive. More than 50 day...,The MTCN code did not arrive. More than 50 day...,The MTCN code did not arrive. More than 50 day...,The MTCN code did not arrive. More than 50 day...,The MTCN code did not arrive. More than 50 day...,...,According to case 269686\nCan't get to the ban...,Repeat request\nA person with reduced mobility...,"A person with limited mobility, a transfer of ...","The recipient of the aid was injured, cannot m...","A person with reduced mobility, does not have ...","Under number 380962139303, there is no informa...",According to case 179681\n\nThe person has pro...,A person with reduced mobility,a person with a disability cannot receive assi...,Changing the phone number\nMs. Lyubov lives in...
label,Request for information - Assistance timing or...,Request for information - Assistance duration,Request for information - Other (please explain),Request for information - Other (please explain),Complaint - Incorrect records,Complaint - Incorrect records,Complaint - Incorrect records,Complaint - Incorrect records,Complaint - Incorrect records,Complaint - Incorrect records,...,Complaint - Physical access challenges,Complaint - Physical access challenges,Complaint - Physical access challenges,Complaint - Physical access challenges,Complaint - Physical access challenges,Complaint - Incorrect records,Complaint - Physical access challenges,Complaint - Physical access challenges,Complaint - Physical access challenges,Complaint - Incorrect records
exact_incoherence_recommendation,Request for information - Targeting criteria,Request for information - Targeting criteria,Request for information - Targeting criteria,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
approx_incoherence_recommendation,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,...,None,None,None,None,None,None,None,None,None,None
model_incoherence_recommendation,None,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Assistance / Delivery Mechanisms /...,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
cluster_incoherence_count,576.0,576.0,576.0,576.0,576.0,576.0,576.0,576.0,576.0,576.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
is_incoherent,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
inspe

In [643]:
inspector.inspect()

2023-12-20 19:04:57.740 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:30 - Calculating 'exact_incoherence' conflicts
2023-12-20 19:04:58.226 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:42 -  └ Filtering conflicted cases
2023-12-20 19:04:58.227 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:46 -  └ Getting cluster indices
2023-12-20 19:04:58.228 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:50 -  └ Getting error recommendations
2023-12-20 19:04:58.229 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:57 -  └ Getting error flags
2023-12-20 19:04:58.246 | INFO     | callclarity.error_report.detectors.approx_incoherence:get_errors:57 -  └ Calculating clusters
2023-12-20 19:05:01.104 | INFO     | callclarity.error_report.detectors.approx_incoherence:get_errors:61 -    └ 3,905 clusters were found.
2023-12-20 19:05:01.105 | INFO     | callclar

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/filter.py:907: UserWarning: May not flag all label issues in class: 5, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/filter.py:907: UserWarning: May not flag all label issues in class: 36, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/filter.py:907: UserWarning: May not flag all label issues in class: 37, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/filter.py:907: UserWarning: May not flag all label issues in class: 38, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cle

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/mx/rrg5pwtj0lg09l5rvnz5y6zr0000gn/T/ipykernel_22306/3375630170.py", line 1, in <module>
    inspector.inspect()
  File "/Users/erichsato/Documents/WFP/01_programming/callclarity/callclarity/error_report/inspector.py", line 180, in inspect
    self.assign_priority()
  File "/Users/erichsato/Documents/WFP/01_programming/callclarity/callclarity/error_report/inspector.py", line 122, in assign_priority
    df["priority"] = df.apply(
  File "/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/pandas/core/frame.py", line 9568, in apply
    return op.apply().__finalize__(self, method="apply")
  File "/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/pandas/core/apply.py", line 764, in apply
    return self.apply_st

In [636]:
_ = inspector.get_errors()

2023-12-20 18:56:11.003 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:30 - Calculating 'exact_incoherence' conflicts
2023-12-20 18:56:11.546 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:42 -  └ Filtering conflicted cases
2023-12-20 18:56:11.548 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:46 -  └ Getting cluster indices
2023-12-20 18:56:11.549 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:50 -  └ Getting error recommendations
2023-12-20 18:56:11.550 | INFO     | callclarity.error_report.detectors.exact_incoherence:get_errors:57 -  └ Getting error flags
2023-12-20 18:56:11.572 | INFO     | callclarity.error_report.detectors.approx_incoherence:get_errors:57 -  └ Calculating clusters
2023-12-20 18:56:14.378 | INFO     | callclarity.error_report.detectors.approx_incoherence:get_errors:61 -    └ 3,905 clusters were found.
2023-12-20 18:56:14.379 | INFO     | callclar

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/filter.py:907: UserWarning: May not flag all label issues in class: 0, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/filter.py:907: UserWarning: May not flag all label issues in class: 1, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/filter.py:907: UserWarning: May not flag all label issues in class: 2, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanlab/filter.py:907: UserWarning: May not flag all label issues in class: 3, it has too few examples (see `min_examples_per_class` argument)
  warnings.warn(
/Users/erichsato/miniconda3/envs/nlp_cfm/lib/python3.10/site-packages/cleanl

In [640]:
df_inspect = inspector.output_df.copy()
df_inspect["_approx_incoherence_cluster_index"] = df_inspect["approx_incoherence_metadata"].str["approx_incoherence_cluster_index"]

In [641]:
df_inspect.head()

,id,text,label,embedding,is_exact_incoherence,exact_incoherence_recommendation,exact_incoherence_metadata,is_approx_incoherence,approx_incoherence_recommendation,approx_incoherence_metadata,is_model_incoherence,model_incoherence_recommendation,model_incoherence_metadata,_approx_incoherence_cluster_index
0,482666,Clarification of the status of the application for eHelp.\n,Request for information - Other (please explain),"[0.02063043601810932, -0.014633174985647202, 0.0018570597749203444, 0.03578193113207817, 0.026378776878118515, 0.010056345723569393, -0.0037040424067527056, 0.015377209521830082, -0.0061657060869038105, 0.00020253939146641642, -0.007710890844464302, -0.019451137632131577, 0.018473321571946144, -0.0076651484705507755, -0.030224254354834557, -0.0013894927687942982, 0.03242503106594086, -0.039075255393981934, -0.016736697405576706, 0.03644832968711853, 0.026604976505041122, -0.029906943440437317, -0.03138010576367378, -0.03566896170377731, -0.005968381650745869, 0.007034401874989271, -0.02914425916969776, -0.03436632081866264, 0.042137566953897476, -0.03530101105570793, 0.025876149535179138, 0.002100934274494648, 0.028253639116883278, -0.018664270639419556, -0.05479830130934715, -0.024769887328147888, 0.05613785237073898, 0.0020175911486148834, 0.0023002440575510263, -0.007379493210464716, 0.08874393254518509, -0.01469328161329031, 0.025371013209223747, 0.015361685305833817, -0.00976340938359499, 0.004464337602257729, 0.044284045696258545, 0.0337451808154583, 0.012728153727948666, 0.0142359659075737, 0.036281414330005646, 0.03620985150337219, 0.014675529673695564, 0.02287684753537178, 0.02511700987815857, -0.007940569892525673, -0.03099764510989189, 0.023507051169872284, -0.020254170522093773, -0.0014974616933614016, 0.028966784477233887, -0.04963718727231026, -0.01459343358874321, 0.02146773971617222, 0.018954424187541008, -0.046064842492341995, 0.03481094539165497, 0.03793967142701149, -0.05907777324318886, -0.0018728224094957113, 0.06972736120223999, 0.08613918721675873, -0.020117398351430893, -0.032251615077257156, 0.004387949127703905, -0.020590726286172867, -0.03717455640435219, -0.048438120633363724, -0.06928769499063492, 0.02950315922498703, -0.01594865694642067, 0.007081903517246246, 0.03608689457178116, 0.04165271297097206, 0.004951501730829477, 0.04289400950074196, -0.03444371744990349, -0.012347990646958351, 0.03336329758167267, -0.005095571745187044, 0.007351362146437168, -0.08307232707738876, 0.021938584744930267, 0.03161279112100601, 0.05435875803232193, -0.013639518059790134, -0.07568836957216263, 0.003482514526695013, -0.020995959639549255, -0.023884855210781097, ...]",0,None,"{'exact_incoherence_cluster_index': nan, 'exact_incoherence_cluster_size': nan, 'exact_incoherence_cluster_candidates': nan}",0,None,"{'approx_incoherence_cluster_index': 1.0, 'approx_incoherence_cluster_size': 1323.0, 'approx_incoherence_cluster_candidates': 60.0}",0,None,{'model_incoherence_severity': nan},1.0
1,482736,The MTCN code did not arrive. Less than 50 days have passed since the last SMS with the code. It is recommended to wait.\n,Request for information - Assistance timing or location,"[-0.008439027704298496, -0.02939574420452118, 0.010878953151404858, 0.011462884955108166, -0.017010273411870003, -0.02308790199458599, -0.02591068483889103, 0.017643386498093605, -0.021967783570289612, -0.007430138532072306, -0.04495438560843468, 0.02255563996732235, 0.003848706604912877, -0.08500702679157257, -0.03630180284380913, -0.06978219747543335, 0.02096705511212349, 0.03162963688373566, 0.012953582219779491, 0.07118736207485199, 0.0015393586363643408, -0.04586445167660713, -0.06180022656917572, -0.0033178257290273905, 0.009632723405957222, 0.010354195721447468, -0.07657653093338013, -0.020427223294973373, 0.05291944369673729, -0.02524198591709137, -0.014135745353996754, 0.00884307362139225, -0.008973411284387112, -0.035615332424640656, 0.03457825258374214, -0.009590371511876583, -0.006010080222040415, -0.005752519704401493,

In [ ]:
inspector.output_df.head()

In [637]:
_ = inspector.get_priority()

AttributeError: 'IncoherenceInspector' object has no attribute 'get_priority'

In [482]:
inspector.output_df["priority"].value_counts().sort_index()

0.0    364
1.0    837
2.0    667
3.0    344
Name: priority, dtype: int64

In [412]:
inspector.detectors[2].model

CleanLearning(clf=LogisticRegression(max_iter=400),
              find_label_issues_kwargs={'confident_joint': array([[  1,   0,   0, ...,   0,   0,   0],
       [  0,   1,   0, ...,   0,   0,   0],
       [  0,   0,   1, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   1,   0,   0],
       [  0,   0,   0, ...,   1,  19,  10],
       [  0,   0,   0, ...,  39,  31, 423]]),
                                        'min_examples_per_class': 10})

## Semantic ordering

In [559]:
nn_graph = inspector.detectors[1].nn_graph
coo_graph = nn_graph.tocoo()
non_zero_cond = (coo_graph.data > 0)

max_embedding_distance = inspector.detectors[1].max_embedding_distance
step = 0.005
sort_steps = np.arange(step, max_embedding_distance + step, step)

sort_matrix = np.zeros(shape = (nn_graph.shape[0],len(sort_steps)))
for index,sort_step in enumerate(sort_steps):
    
    # filter only connections that matter
    #   1. it has to be connected (non zero)
    #   2. it has to be sorted
    mask = non_zero_cond & (coo_graph.data <= sort_step)
    filtered_nn_graph = csr_matrix((coo_graph.data[mask], (coo_graph.row[mask], coo_graph.col[mask])), shape=nn_graph.shape)
    
    # calculate calculated components
    n_components, labels = connected_components(filtered_nn_graph)
    
    # sort and reindex, assign smaller indices for larger components
    #   1. create count dictionary
    #   2. reindex components according to size
    #   3. reassign components indices
    element_counts = dict(zip(*np.unique(labels, return_counts=True)))
    element_counts = dict(sorted(element_counts.items(), key = lambda item: item[1], reverse = True))
    index_map = {
        old_index:new_index
        for new_index, old_index in enumerate(element_counts.keys())
    }
    labels = np.array([index_map[old_index] for old_index in labels])
    
    # store indices in sort matrix
    sort_matrix[:,index] = labels

# create indices sorted by multiple columns, hierarchically. 
sorted_indices = np.lexsort(sort_matrix.T, axis = 0)

In [568]:
df_test.iloc[sorted_indices]

,id,text,label,is_exact_incoherence,exact_incoherence_recommendation,exact_incoherence_metadata,is_approx_incoherence,approx_incoherence_recommendation,approx_incoherence_metadata,is_model_incoherence,model_incoherence_recommendation,model_incoherence_metadata,priority
450,495778,The MTCN code did not arrive. More than 50 days since the last SMS with the code. MTCN resubmission required.\n,Complaint - Assistance / Delivery Mechanisms / Payment Instruments,0,None,"{'exact_incoherence_cluster_index': nan, 'exact_incoherence_cluster_size': nan, 'exact_incoherence_cluster_candidates': nan}",0,None,"{'approx_incoherence_cluster_index': 0.0, 'approx_incoherence_cluster_size': 1393.0, 'approx_incoherence_cluster_candidates': 576.0}",0,None,{'model_incoherence_severity': nan},NaN
524,497983,The MTCN code did not arrive. More than 50 days since the last SMS with the code. MTCN resubmission required.\n,Complaint - Assistance / Delivery Mechanisms / Payment Instruments,0,None,"{'exact_incoherence_cluster_index': nan, 'exact_incoherence_cluster_size': nan, 'exact_incoherence_cluster_candidates': nan}",0,None,"{'approx_incoherence_cluster_index': 0.0, 'approx_incoherence_cluster_size': 1393.0, 'approx_incoherence_cluster_candidates': 576.0}",0,None,{'model_incoherence_severity': nan},NaN
533,498099,The MTCN code did not arrive. More than 50 days since the last SMS with the code. MTCN resubmission required.\n,Complaint - Assistance / Delivery Mechanisms / Payment Instruments,0,None,"{'exact_incoherence_cluster_index': nan, 'exact_incoherence_cluster_size': nan, 'exact_incoherence_cluster_candidates': nan}",0,None,"{'approx_incoherence_cluster_index': 0.0, 'approx_incoherence_cluster_size': 1393.0, 'approx_incoherence_cluster_candidates': 576.0}",0,None,{'model_incoherence_severity': nan},NaN
662,502173,The MTCN code did not arrive. More than 50 days since the last SMS with the code. MTCN resubmission required.,Complaint - Assistance / Delivery Mechanisms / Payment Instruments,0,None,"{'exact_incoherence_cluster_index': nan, 'exact_incoherence_cluster_size': nan, 'exact_incoherence_cluster_candidates': nan}",0,None,"{'approx_incoherence_cluster_index': 0.0, 'approx_incoherence_cluster_size': 1393.0, 'approx_incoherence_cluster_candidates': 576.0}",0,None,{'model_incoherence_severity': nan},NaN
696,503377,The MTCN code did not arrive. More than 50 days since the last SMS with the code. MTCN resubmission required.,Complaint - Assistance / Delivery Mechanisms / Payment Instruments,0,None,"{'exact_incoherence_cluster_index': nan, 'exact_incoherence_cluster_size': nan, 'exact_incoherence_cluster_candidates': nan}",0,None,"{'approx_incoherence_cluster_index': 0.0, 'approx_incoherence_cluster_size': 1393.0, 'approx_incoherence_cluster_candidates': 576.0}",0,None,{'model_incoherence_severity': nan},NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10327,552764,A person with reduced mobility,Complaint - Physical access challenges,0,None,"{'exact_incoherence_cluster_index': nan, 'exact_incoherence_cluster_size': nan, 'exact_incoherence_cluster_candidates': nan}",0,None,"{'approx_incoherence_cluster_index': nan, 'approx_incoherence_cluster_size': nan, 'approx_incoherence_cluster_candidates': nan}",0,None,{'model_incoherence_severity': nan},NaN
10328,537334,a person with a disability cannot receive assistance,Complaint - Physical access challenges,0,None,"{'exact_incoherence_cluster_index': nan, 'exact_incoherence_cluster_size': nan, 'exact_incoherence_cluster_candidates': nan}",0,None,"{'approx_incoherence_cluster_index': nan, 'approx_incoherence_cluster_size': nan, 'approx_incoherence_cluster_candidates': nan}",0,None,{'model_incoherence_severity': nan},NaN
10329,538015,"Text of case number 229107\nUnable to receive transfer. Man is not walking. The card cannot be received - it is refused.\nPlease consider the issue and forward the MTSN code that arrived on November 17\nmade a bank card. received 1 payment. the code is expired for 2 previous

In [529]:
df_test = inspector.output_df.drop(columns = ["embedding"]).copy()

In [561]:
df_test.iloc[[-1,0]]

,id,text,label,is_exact_incoherence,exact_incoherence_recommendation,exact_incoherence_metadata,is_approx_incoherence,approx_incoherence_recommendation,approx_incoherence_metadata,is_model_incoherence,model_incoherence_recommendation,model_incoherence_metadata,priority
10339,558435,The MTCN code did not arrive. More than 50 days since the last SMS with the code. MTCN resubmission required.\n,Complaint - Assistance / Delivery Mechanisms / Payment Instruments,0,None,"{'exact_incoherence_cluster_index': nan, 'exact_incoherence_cluster_size': nan, 'exact_incoherence_cluster_candidates': nan}",0,None,"{'approx_incoherence_cluster_index': 0.0, 'approx_incoherence_cluster_size': 1393.0, 'approx_incoherence_cluster_candidates': 576.0}",0,None,{'model_incoherence_severity': nan},NaN
0,482666,Clarification of the status of the application for eHelp.\n,Request for information - Other (please explain),0,None,"{'exact_incoherence_cluster_index': nan, 'exact_incoherence_cluster_size': nan, 'exact_incoherence_cluster_candidates': nan}",0,None,"{'approx_incoherence_cluster_index': 1.0, 'approx_incoherence_cluster_size': 1323.0, 'approx_incoherence_cluster_candidates': 60.0}",0,None,{'model_incoherence_severity': nan},NaN


In [531]:
df_test.sort_values(sort_matrix)

KeyError: array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 1.000e+00, 1.000e+00,
        1.000e+00],
       [3.000e+00, 3.000e+00, 3.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [3.000e+00, 3.000e+00, 3.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [5.612e+03, 5.454e+03, 5.326e+03, ..., 9.000e+00, 1.000e+01,
        1.000e+01],
       [5.613e+03, 5.455e+03, 5.327e+03, ..., 4.314e+03, 4.093e+03,
        3.904e+03],
       [1.000e+00, 1.000e+00, 1.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00]])

In [550]:
sorted_indices = np.lexsort(sort_matrix.T, axis = 0)

In [551]:
df_test.iloc[sorted_indices]

,id,text,label,is_exact_incoherence,exact_incoherence_recommendation,exact_incoherence_metadata,is_approx_incoherence,approx_incoherence_recommendation,approx_incoherence_metadata,is_model_incoherence,model_incoherence_recommendation,model_incoherence_metadata,priority
450,495778,The MTCN code did not arrive. More than 50 days since the last SMS with the code. MTCN resubmission required.\n,Complaint - Assistance / Delivery Mechanisms / Payment Instruments,0,None,"{'exact_incoherence_cluster_index': nan, 'exact_incoherence_cluster_size': nan, 'exact_incoherence_cluster_candidates': nan}",0,None,"{'approx_incoherence_cluster_index': 0.0, 'approx_incoherence_cluster_size': 1393.0, 'approx_incoherence_cluster_candidates': 576.0}",0,None,{'model_incoherence_severity': nan},NaN
524,497983,The MTCN code did not arrive. More than 50 days since the last SMS with the code. MTCN resubmission required.\n,Complaint - Assistance / Delivery Mechanisms / Payment Instruments,0,None,"{'exact_incoherence_cluster_index': nan, 'exact_incoherence_cluster_size': nan, 'exact_incoherence_cluster_candidates': nan}",0,None,"{'approx_incoherence_cluster_index': 0.0, 'approx_incoherence_cluster_size': 1393.0, 'approx_incoherence_cluster_candidates': 576.0}",0,None,{'model_incoherence_severity': nan},NaN
533,498099,The MTCN code did not arrive. More than 50 days since the last SMS with the code. MTCN resubmission required.\n,Complaint - Assistance / Delivery Mechanisms / Payment Instruments,0,None,"{'exact_incoherence_cluster_index': nan, 'exact_incoherence_cluster_size': nan, 'exact_incoherence_cluster_candidates': nan}",0,None,"{'approx_incoherence_cluster_index': 0.0, 'approx_incoherence_cluster_size': 1393.0, 'approx_incoherence_cluster_candidates': 576.0}",0,None,{'model_incoherence_severity': nan},NaN
662,502173,The MTCN code did not arrive. More than 50 days since the last SMS with the code. MTCN resubmission required.,Complaint - Assistance / Delivery Mechanisms / Payment Instruments,0,None,"{'exact_incoherence_cluster_index': nan, 'exact_incoherence_cluster_size': nan, 'exact_incoherence_cluster_candidates': nan}",0,None,"{'approx_incoherence_cluster_index': 0.0, 'approx_incoherence_cluster_size': 1393.0, 'approx_incoherence_cluster_candidates': 576.0}",0,None,{'model_incoherence_severity': nan},NaN
696,503377,The MTCN code did not arrive. More than 50 days since the last SMS with the code. MTCN resubmission required.,Complaint - Assistance / Delivery Mechanisms / Payment Instruments,0,None,"{'exact_incoherence_cluster_index': nan, 'exact_incoherence_cluster_size': nan, 'exact_incoherence_cluster_candidates': nan}",0,None,"{'approx_incoherence_cluster_index': 0.0, 'approx_incoherence_cluster_size': 1393.0, 'approx_incoherence_cluster_candidates': 576.0}",0,None,{'model_incoherence_severity': nan},NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10327,552764,A person with reduced mobility,Complaint - Physical access challenges,0,None,"{'exact_incoherence_cluster_index': nan, 'exact_incoherence_cluster_size': nan, 'exact_incoherence_cluster_candidates': nan}",0,None,"{'approx_incoherence_cluster_index': nan, 'approx_incoherence_cluster_size': nan, 'approx_incoherence_cluster_candidates': nan}",0,None,{'model_incoherence_severity': nan},NaN
10328,537334,a person with a disability cannot receive assistance,Complaint - Physical access challenges,0,None,"{'exact_incoherence_cluster_index': nan, 'exact_incoherence_cluster_size': nan, 'exact_incoherence_cluster_candidates': nan}",0,None,"{'approx_incoherence_cluster_index': nan, 'approx_incoherence_cluster_size': nan, 'approx_incoherence_cluster_candidates': nan}",0,None,{'model_incoherence_severity': nan},NaN
10329,538015,"Text of case number 229107\nUnable to receive transfer. Man is not walking. The card cannot be received - it is refused.\nPlease consider the issue and forward the MTSN code that arrived on November 17\nmade a bank card. received 1 payment. the code is expired for 2 previous

# Finalizing the report

In [40]:
import cleanlab

In [45]:
cl = inspector.detectors[2].model

In [50]:
classes_problems = cleanlab.dataset.find_overlapping_classes(
    labels=inspector.detectors[2].labels,
    confident_joint=cl.confident_joint,  # cleanlab uses the confident_joint internally to quantify label noise (see cleanlab.count.compute_confident_joint)
    class_names=inspector.detectors[2].label_encoder.classes_,
)

In [54]:
classes_problems[classes_problems["Joint Probability"] > 0.001]

,Class Name A,Class Name B,Class Index A,Class Index B,Num Overlapping Examples,Joint Probability
0,Request for information - Other organization a...,Request for information - Targeting criteria,33,35,219,0.021180
1,Request for information - Other (please explain),Request for information - Targeting criteria,32,35,136,0.013153
2,Request for information - Assistance timing or...,Request for information - Targeting criteria,29,35,119,0.011509
3,Request for information - Other (please explain),Request for information - Other organization a...,32,33,83,0.008027
4,Request for information - Assistance timing or...,Request for information - Other organization a...,29,33,78,0.007544
5,Request for information - Assistance duration,Request for information - Targeting criteria,28,35,68,0.006576
6,Complaint - Assistance / Delivery Mechanisms /...,Request for information - Targeting criteria,6,35,67,0.006480
7,Complaint - Assistance / Delivery Mechanisms /...,Request for information - Other (please explain),6,32,56,0.005416
8,Interaction Issues - Interrupted interaction,Interaction Issues - Other (please explain),18,19,56,0.005416
9,Complaint - Assistance / Delivery Mechanisms /...,Complaint - Incorrect records,6,8,54,0.005222


In [55]:
health = cleanlab.dataset.overall_label_health_score(
    inspector.detectors[2].labels, confident_joint=cl.confident_joint
    # cleanlab uses the confident_joint internally to quantify label noise (see cleanlab.count.compute_confident_joint)
)

 * Overall, about 21% (2,202 of the 10,340) labels in your dataset have potential issues.
 ** The overall label health score for this dataset is: 0.79.
